In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)


In [ ]:

from dotenv import load_dotenv

from langchain_community.utilities import GoogleSerperAPIWrapper

from autogen_ext.tools.http import HttpTool


os.environ['SERPER_API_KEY']='bead05022450578faa7498f4c90d85e534c372e0'


search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."  



In [ ]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [ ]:
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [ ]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [ ]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100



In [ ]:

data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

# Termination Condition

In [ ]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [ ]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [ ]:
planning_agent.description

In [ ]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [ ]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [ ]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

In [ ]:
# With real web search


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

In [ ]:
state = await selector_team.save_state()

In [ ]:
state


In [ ]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source == web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)

In [17]:
# With real web search
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your query, we need to follow these steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Determine this player's total rebounds in the 2007-2008 season.
3. Determine this player's total rebounds in the 2008-2009 season.
4. Calculate the percentage change in total rebounds between these two seasons.

Let's delegate these tasks:

1. WebSearchAgent: Identify the Miami Heat player with the most points in the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for that player in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds for that player in the 2008-2009 season.

Once these details are collected:

4. DataAnalystAgent: Calculate the percentage c

TaskResult(messages=[TextMessage(id='c5329421-955f-4c45-aae5-c043ceeaf71b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 0, 53, 147954, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c4795a65-7e77-4e88-a73d-6a3c22b9144c', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=194), metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 0, 55, 864264, tzinfo=datetime.timezone.utc), content="To address your query, we need to follow these steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Determine this player's total rebounds in the 2007-2008 season.\n3. Determine this player's total rebounds in the 2008-2009 season.\n4. Calculate the percentage change i

In [23]:
await selector_team.reset()

In [25]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False,
    selector_func=my_selector_fun)


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (DataAnalystAgent) ----------
Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. His total rebounds decreased by approximately 18.56% from the 2007-2008 season (334 rebounds) to the 2008-2009 season (272 rebounds).
---------- TextMessage (DataAnalystAgent) ----------
Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. His total rebounds decreased by approximately 18.56% from the 2007-2008 season (334 rebounds) to the 2008-2009 season (272 rebounds).
---------- TextMessage (DataAnalystAgent) ----------
Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. His total rebounds decreased by approximately 18.56% from the 2007-2008 season (334 rebounds) to the 2008-2009 season (272 rebounds).
---------- TextMessage (DataAnalystAgent) ----------
Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. His total rebounds decreased by approx

TaskResult(messages=[TextMessage(id='b7c06dcd-af8e-4946-b773-6f7b140cef2f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 9, 34, 943116, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='f0a1def5-8340-4c91-a08e-9542f80e68b6', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=612, completion_tokens=61), metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 9, 36, 27638, tzinfo=datetime.timezone.utc), content='Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. His total rebounds decreased by approximately 18.56% from the 2007-2008 season (334 rebounds) to the 2008-2009 season (272 rebounds).', type='TextMessage'), TextMessage(id='68a29755-ec97-4207-aeb5-abb0f035eda3', source='DataA

In [ ]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


In [26]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    

    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    

    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
        

    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
# await sele.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))


---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address this, let's proceed with the necessary tasks:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for the identified player in both the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in total rebounds for the player between the two seasons.
---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_AykDDUYLwJV6df7XUfUUpCsC', arguments='{"query": "Miami Heat player total rebounds 2007-2008 and 2008-2009 seasons Dwyane Wade"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent)

TaskResult(messages=[TextMessage(id='0bb3e17d-787a-4b8b-8efd-4e206847b251', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 26, 53, 447296, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='49652a73-383d-4ca9-9d6c-1d3781ca6ffe', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=640, completion_tokens=90), metadata={}, created_at=datetime.datetime(2025, 7, 19, 3, 26, 54, 834484, tzinfo=datetime.timezone.utc), content="To address this, let's proceed with the necessary tasks:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Find the total rebounds for the identified player in both the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the pe